The spread and intensity of Covid-19 is influenced by development levels of a country. This file traces the impact of Covid-19 on countries with different levels of development.

The countries are divided into 4 categories(based on UNDP definition)-
1. Very High Human Development
2. High Human Development
3. Medium Human Development
4. Low Human Development

The parameters on which these countries are compared-
    * total cases per million      * deaths per hundred cases
    * hospital beds per thousand   * GDP per capita
    * poverty rate
    
Trends are shown through interactive maps. Hover over the graph to see the values for each country.

For more related content, visit my github- https://github.com/AnuragPatel101/Covid-19

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import plotly as py
py.offline.init_notebook_mode(connected = True)

import plotly.graph_objects as go
import folium

In [2]:
df = pd.read_csv('Covid_19.csv')
df.shape

(51381, 21)

In [3]:
# checking null values
df.isnull().sum()[df.isnull().sum() > 0]

Series([], dtype: int64)

In [4]:
# no null values

In [5]:
# checking features
df.columns

Index(['date', 'country', 'new_cases', 'new_deaths', 'total_cases',
       'total_deaths', 'weekly_cases', 'weekly_deaths', 'biweekly_cases',
       'biweekly_deaths', 'continent', 'total_cases_per_million',
       'total_deaths_per_million', 'population', 'gdp_per_capita',
       'extreme_poverty', 'hospital_beds_per_thousand', 'life_expectancy',
       'human_development_index', 'Lat', 'Long'],
      dtype='object')

In [6]:
# number of unique countries
df['country'].nunique()

211

In [7]:
# getting index of unique countries
first_index = [0]
ind = 0
for i in range(210):
    for j in range(first_index[ind], 51381):
        if(df.loc[j, 'country'] != df.loc[first_index[ind], 'country']):
            first_index.append(j)
            ind += 1
            break

In [8]:
# first_index has first index of each unique country

In [9]:
last_index = np.array(first_index[1:])-1
last_index = list(last_index)
last_index.append(df.shape[0]-1)

In [10]:
# last_index has last index of all unique countries

In [11]:
# creating dataframe for this file
dfn = df.loc[last_index][['country', 'continent', 'Lat', 'Long', 'population', 'total_cases_per_million',
                          'total_deaths_per_million', 'hospital_beds_per_thousand', 'gdp_per_capita',
                          'extreme_poverty', 'human_development_index']]
dfn.head(2)

,country,continent,Lat,Long,population,total_cases_per_million,total_deaths_per_million,hospital_beds_per_thousand,gdp_per_capita,extreme_poverty,human_development_index
297,Afghanistan,Asia,33.0000,65.0000,38928341.0,1043.61,38.661,0.50,1803.987,42.0,0.498
526,Albania,Europe,41.1533,20.1683,2877800.0,6341.65,161.582,2.89,11803.431,1.1,0.785


In [12]:
# resetting the index
dfn.reset_index(drop = True, inplace = True)

In [13]:
# deaths per hundred cases
dfn['deaths_per_hundred_cases'] = (dfn['total_deaths_per_million']/dfn['total_cases_per_million'])*100

In [14]:
# Human Development Classification by UNDP
# 0.4 <= hdi < 0.55 -> low human development -> LD
# 0.55 <= hdi < 0.70 -> medium human development -> MD
# 0.70 <= hdi < 0.80 -> high human development -> HD
# 0.80 <= hdi -> very high human development -> VHD

In [15]:
# categorizing countries on basis of human development
hdi_cat = []
for i in range(211):
    if(dfn.loc[i, 'human_development_index'] < 0.55):
        hdi_cat.append('LD')
    elif(0.55 <= dfn.loc[i, 'human_development_index'] < 0.70):
        hdi_cat.append('MD')
    elif(0.70 <= dfn.loc[i, 'human_development_index'] < 0.80):
        hdi_cat.append('HD')
    else:
        hdi_cat.append('VHD')
        
dfn['hdi_cat'] = pd.Series(hdi_cat)
dfn['hdi_cat'].value_counts()

VHD    81
HD     53
LD     39
MD     38
Name: hdi_cat, dtype: int64

### Plot of total cases per million for different categories of countries

In [16]:
clr_avg = ['Darkgreen', 'Blue', 'Brown', 'Red'] # colour palette for average cases per million
clr_tcm = ['Lightgreen', 'Skyblue', 'Crimson', 'Pink'] # colour palette for total cases per million
clr_ind = 0 # colour index

dict_devt = {'VHD' : 'Very High Development', 'HD' : 'High Development', 'MD' : 'Medium Development', 'LD' : 'Low Development'}

for i in ['VHD', 'HD', 'MD', 'LD']:
    avg = dfn[dfn['hdi_cat'] == i]['total_cases_per_million'].mean()
    avg_ser = pd.Series([avg]*(dfn[dfn['hdi_cat'] == i].shape[0]))
    fig = go.Figure()
    fig.add_trace(go.Scatter(x = dfn[dfn['hdi_cat'] == i]['country'], y = dfn[dfn['hdi_cat'] == i]['total_cases_per_million'],
                             name = 'total cases per million', mode = 'lines+markers', line = dict(color = clr_tcm[clr_ind], width = 1.5)))
    fig.add_trace(go.Scatter(x = dfn[dfn['hdi_cat'] == i]['country'], y = avg_ser,
                             name = 'average cases per million', mode = 'lines', line = dict(color = clr_avg[clr_ind], width = 1.5)))
    fig.update_layout(title = dict_devt[i])
    clr_ind += 1
    fig.show()

**Average total cases per million**
    * Very High Development -> 12k
    * High Development      -> 7.03k
    * Medium Development    -> 4.36k
    * Low Development       -> 0.6k

We can see that the less developed countries have lower cases per million. The possible reasons could be -
1. High population in less developed countries
2. Low capacity of less developed countries in collection and reporting of cases
3. Too some extent, treating Covid as a hoax in the start, by developed countries

### Plot of total deaths per hundred cases and hospital beds per thousand for different categories of countries

In [17]:
dict_devt = {'VHD' : 'Very High Development', 'HD' : 'High Development', 'MD' : 'Medium Development', 'LD' : 'Low Development'}

for i in ['VHD', 'HD', 'MD', 'LD']:
    avg = dfn[dfn['hdi_cat'] == i]['deaths_per_hundred_cases'].mean()
    avg_ser = pd.Series([avg]*(dfn[dfn['hdi_cat'] == i].shape[0]))
    fig = go.Figure()
    fig.add_trace(go.Scatter(x = dfn[dfn['hdi_cat'] == i]['country'], y = dfn[dfn['hdi_cat'] == i]['deaths_per_hundred_cases'],
                             name = 'deaths per hundred cases', mode = 'lines+markers', line = dict(color = 'Red', width = 1.5)))
    fig.add_trace(go.Scatter(x = dfn[dfn['hdi_cat'] == i]['country'], y = avg_ser,
                             name = 'average deaths per hundred cases', mode = 'lines', line = dict(color = 'Brown', width = 1.5)))
    fig.update_layout(title = dict_devt[i])
    
    fig.show()
    
    avg = dfn[dfn['hdi_cat'] == i]['hospital_beds_per_thousand'].mean()
    avg_ser = pd.Series([avg]*(dfn[dfn['hdi_cat'] == i].shape[0]))
    fig = go.Figure()
    fig.add_trace(go.Scatter(x = dfn[dfn['hdi_cat'] == i]['country'], y = dfn[dfn['hdi_cat'] == i]['hospital_beds_per_thousand'],
                             name = 'hospital beds per thousand', mode = 'lines+markers', line = dict(color = 'Darkgreen', width = 1.5)))
    fig.add_trace(go.Scatter(x = dfn[dfn['hdi_cat'] == i]['country'], y = avg_ser,
                             name = 'average hospital beds per thousand', mode = 'lines', line = dict(color = 'Lightgreen', width = 1.5)))
    fig.update_layout(title = dict_devt[i])
    fig.show()
    print('--------------------------------------------------------------------------------------------------------------------')

--------------------------------------------------------------------------------------------------------------------


--------------------------------------------------------------------------------------------------------------------


--------------------------------------------------------------------------------------------------------------------


--------------------------------------------------------------------------------------------------------------------


**Average deaths per hundred cases**
    * Very High Development -> 2.13
    * High Development      -> 2.7
    * Medium Development    -> 2.22
    * Low Development       -> 3.95
    
**Average hospital beds per thousand**
    * Very High Development -> 4.34
    * High Development      -> 3.16
    * Medium Development    -> 1.72
    * Low Development       -> 0.83

The death rate is high for Low Human Development countries. One of the reasons is low availability of beds.

There is also need to look at other indicators like GDP per capita and Extreme Poverty.

### Plot of GDP per capita and Extreme Poverty for different categories of countries

In [18]:
dict_devt = {'VHD' : 'Very High Development', 'HD' : 'High Development', 'MD' : 'Medium Development', 'LD' : 'Low Development'}

for i in ['VHD', 'HD', 'MD', 'LD']:
    avg = dfn[dfn['hdi_cat'] == i]['gdp_per_capita'].mean()
    avg_ser = pd.Series([avg]*(dfn[dfn['hdi_cat'] == i].shape[0]))
    fig = go.Figure()
    fig.add_trace(go.Scatter(x = dfn[dfn['hdi_cat'] == i]['country'], y = dfn[dfn['hdi_cat'] == i]['gdp_per_capita'],
                             name = 'GDP per capita', mode = 'lines+markers', line = dict(color = 'Green', width = 1.5)))
    fig.add_trace(go.Scatter(x = dfn[dfn['hdi_cat'] == i]['country'], y = avg_ser,
                             name = 'average GDP per capita', mode = 'lines', line = dict(color = 'Orange', width = 1.5)))
    fig.update_layout(title = dict_devt[i])
    
    fig.show()
    
    avg = dfn[dfn['hdi_cat'] == i]['extreme_poverty'].mean()
    avg_ser = pd.Series([avg]*(dfn[dfn['hdi_cat'] == i].shape[0]))
    fig = go.Figure()
    fig.add_trace(go.Scatter(x = dfn[dfn['hdi_cat'] == i]['country'], y = dfn[dfn['hdi_cat'] == i]['extreme_poverty'],
                             name = 'poverty rate', mode = 'lines+markers', line = dict(color = 'Magenta', width = 1.5)))
    fig.add_trace(go.Scatter(x = dfn[dfn['hdi_cat'] == i]['country'], y = avg_ser,
                             name = 'average poverty rate', mode = 'lines', line = dict(color = 'Darkblue', width = 1.5)))
    fig.update_layout(title = dict_devt[i])
    fig.show()
    print('--------------------------------------------------------------------------------------------------------------------')

--------------------------------------------------------------------------------------------------------------------


--------------------------------------------------------------------------------------------------------------------


--------------------------------------------------------------------------------------------------------------------


--------------------------------------------------------------------------------------------------------------------


**Average GDP per capita**
    * Very High Development -> 42.85k
    * High Development      -> 15.14k
    * Medium Development    -> 7.42k
    * Low Development       -> 1.96k
    
**Average poverty rate**
    * Very High Development -> 6.88
    * High Development      -> 10.21
    * Medium Development    -> 18.78
    * Low Development       -> 45.95

To a large extent, GDP per capita has positive correlation with high development. This allows for better health facilities and better access to them in developed world.

To a large extent, poverty has negative correlation with development. High poverty rates result in poor health standards like high prevealnce of malnutrition and diseases.

GDP per capita and poverty has high influence on death rates.